# Sherlock 

> Python SDK for interacting with the Sherlock Domains API.

In [ ]:
#| default_exp sherlock

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from typing import Optional, List

from fastcore.test import *
from sherlock_domains.keys import load_key
from sherlock_domains import api

In [ ]:
#| export
class Sherlock:
    """Sherlock is an SDK for interacting with the Sherlock Domains API."""

    API_URL = "https://api.sherlockdomains.com"

    def __init__(self, private_key_hex: Optional[str] = None):
        if private_key_hex is not None:
            self.key = load_key(private_key_hex)
            self._authenticate()
    
    def _authenticate(self):
        """Authenticate with the Sherlock API"""
        if self.key is None:
            raise ValueError("Unable to authenticate: no private key provided")
        
        pub_key_hex = self.key.public_key().hex()
        challenge = api.get_new_challenge(pub_key_hex)
        signature = self.key.sign(bytes.fromhex(challenge))
        tokens = api.get_tokens(pub_key_hex, challenge, signature.hex())
        
        self.access_token = tokens["access_token"]
        self.refresh_token = tokens["refresh_token"]
        self._is_authenticated = True

    def _auth_token(self) -> str:
        """Get the access token"""
        if not self._is_authenticated:
            raise ValueError("Unable to get access token: not authenticated")
        
        # TODO: handle token expiration
        return self.access_token

    def search_domains(self, query: str) -> api.DomainSearchResponse:
        return api.search_domains(self._auth_token(), query)
    
    def purchase_domain(self, search_id: str, domain: str, contact_info: api.ContactInformation, payment_method: str = "credit_card") -> str:
        domain_offer = api.request_offer(search_id, domain, contact_info, self._auth_token())

        if domain_offer is None:
            raise ValueError(f"No offer found for domain: {domain}")
        
        payment_request_url = domain_offer["payment_request_url"]
        payment_context_token = domain_offer["payment_context_token"]
        offer_id = domain_offer["offers"][0]["id"]


        payment_request = api.get_payment_request(payment_request_url, payment_context_token, offer_id, payment_method)
        if payment_method == "credit_card":
            return payment_request["payment_method"]["checkout_url"]
        elif payment_method == "lightning":
            return payment_request["payment_method"]["lightning_invoice"]

    def get_domains(self) -> List[api.DomainInfo]:
        return api.get_domains(self._auth_token())
    
    def get_dns_records(self, domain: str) -> List[api.DNSRecord]:
        return api.get_dns_records(self._auth_token(), domain)
    
    def create_dns_record(self, domain: str, record: api.CreateDNSRecord) -> api.DNSRecord:
        return api.create_dns_record(self._auth_token(), domain, record)
    
    def update_dns_record(self, domain: str, record_id: str, record: api.UpdateDNSRecord) -> api.DNSRecord:
        return api.update_dns_record(self._auth_token(), domain, record_id, record)
    
    def delete_dns_record(self, domain: str, record_id: str) -> None:
        return api.delete_dns_record(self._auth_token(), domain, record_id)


AttributeError: module 'sherlock_domains.api' has no attribute 'ContactInfo'